# Arc Results

In [ ]:
%matplotlib inline
from arc_maps import get_data
from smif.data_layer import Results
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
results = Results({'interface': 'local_csv', 'dir': '../'})

sbn.set_context('talk', rc={"lines.linewidth": 2.5})
sbn.set_palette(sbn.color_palette('Paired'))

In [ ]:
filepath = '../credential.ini'
arc_lads = get_data.get_lads(filepath)

# View the LADs

In [ ]:
def plot_highlighted_lads(arc_lads):
    
    arc_lads = highlight(arc_lads.reset_index())
    
#     arc_lads = arc_lads[~(arc_lads['summary_lads'] == 'Other')]
    
    ax = arc_lads.plot(column='summary_lads')
    
    centroids = [(float(x.x), float(x.y)) for x in list(arc_lads.centroid)]
    for label, coord in zip(arc_lads['summary_lads'].values, centroids):
        if label != 'Other':
            plt.annotate(label, coord)
    ax.set_axis_off()
    
with sbn.plotting_context('paper'):
    plot_highlighted_lads(arc_lads)

In [ ]:
highlight(arc_lads.reset_index())

In [ ]:
results.list_model_runs()

In [ ]:
results.available_results('arc_et__baseline')

In [ ]:
results.list_scenario_outputs('socio-economic')
dwellings = results.read_scenario_data(
    'socio-economic', 'arc_baseline', 'dwellings', [2015, 2020, 2030, 2050])
# sbn.relplot(x='timestep', y='gva_per_sector', hue='sectors', kind='line', data=gva.reset_index())

# Plotting Arc scenario data

The outputs of the socio economic scenarios include:

In [ ]:
results.list_scenario_outputs('socio-economic')

In [ ]:
results.available_results('arc_et__baseline')

In [ ]:
def get_scenario_data(scenario_output):

    arc_lads = get_data.get_lads(filepath)

    data = ['arc_baseline', 'arc_unplanned', 'arc_new-cities', 'arc_expansion']
    result = []

    for name in data:

        result.append(results.read_scenario_data(
        'socio-economic', name, scenario_output, [2015, 2020, 2030, 2050]).set_index(
        ['lad_uk_2016', 'timestep']))

    # Make the initial df from the baseline data
    dwellings = result.pop(0).reset_index().set_index(['lad_uk_2016', 'timestep']).rename(columns={scenario_output: 'arc_baseline'})
    data.pop(0)

    for name, df in zip(data, result):

        df = df.reset_index().set_index(['lad_uk_2016', 'timestep'])
        suffix = "_{}".format(name.split("_")[1])
        dwellings[name] = df[scenario_output]

    arc_dwellings = arc_lads.join(
        dwellings.reset_index().set_index(
        ['lad_uk_2016'])).drop(
        columns=['gid']).rename(
        columns={scenario_output: 'baseline'})
    
    return arc_dwellings

In [ ]:
def reshape(df):
    
    return df.melt(id_vars=['geo_label', 'timestep', 'geometry'], 
                  value_vars=['arc_baseline',  'arc_unplanned', 'arc_new-cities', 'arc_expansion'],
                  var_name='scenario')

def plot_total_scenario_data(df, y_label=None):
    data = reshape(df).groupby(by=['scenario', 'timestep']).sum().reset_index()
    fig = sbn.relplot(x='timestep', y='value', hue='scenario', data=data, kind='line',
                      height=4)
    fig.set_axis_labels('year', y_label)
    return fig
    
def plot_scenario_data(df):
    sbn.set_palette("Paired")
    data = reshape(df)
      
    return sbn.relplot(x='timestep', y='value', hue='geo_label', data=data, col='scenario', kind='line',
                      col_wrap=2)
    
def plot_scenario_data_highlight(df, y_label=None):
    
    data = reshape(df)
    data = highlight(data)   
    fig = sbn.relplot(x='timestep', y='value', hue='summary_lads', 
                      data=data, col='scenario', kind='line', col_wrap=2)
    fig.set_axis_labels('year', y_label)
    return fig

def highlight(data):
    
    def group_hlads(index):
        if index in {'Bedford', 'Cambridge', 'Milton Keynes', 'Oxford', 
                     'Aylesbury Vale', 'Central Bedfordshire', 'Huntingdonshire', 'South Cambridgeshire'}:
            return index
        else:
            return 'Other'
    
    data['summary_lads'] = data['geo_label'].apply(group_hlads) 
    
    return data

def plot_bar_highlight(df, y_label=None):
    
    data = reshape(df)
    data = highlight(data)
    fig = sbn.catplot(x='timestep', y='value', hue='summary_lads', data=data, col='scenario', 
                      kind='bar', col_wrap=2)
    fig.set_axis_labels('year', y_label)
    return fig
   

In [ ]:
arc_dwellings = get_scenario_data('dwellings')
plot_scenario_data(arc_dwellings)

In [ ]:
fig = plot_scenario_data_highlight(arc_dwellings, 'dwellings')
fig.savefig("dwellings_highlight.png")

In [ ]:
plot_bar_highlight(arc_dwellings, 'dwellings')

In [ ]:
arc_population = get_scenario_data('population')
fig = plot_scenario_data_highlight(arc_population, 'people')
fig.savefig("population_highlight.png")

In [ ]:
plot_scenario_data_highlight(arc_population, 'people')

In [ ]:
arc_gva = get_scenario_data('gva')

In [ ]:
arc_gva.head()

In [ ]:
plot_scenario_data(arc_gva)

In [ ]:
plot_total_scenario_data(arc_dwellings, 'dwellings').savefig("total_dwellings.png")

In [ ]:
plot_total_scenario_data(arc_gva, 'gva').savefig("total_gva.png")

In [ ]:
plot_total_scenario_data(arc_population, 'people').savefig("total_population.png")

In [ ]:
arc_jobs = get_scenario_data('employment')

In [ ]:
plot_total_scenario_data(arc_jobs, 'jobs').savefig("total_jobs.png")

In [ ]:
arc_jobs.head(10)

In [ ]:
fig = plot_scenario_data_highlight(arc_jobs)
fig.savefig("jobs_highlight.png")

In [ ]:
df = arc_dwellings.melt(id_vars=['geo_label', 'timestep', 'geometry'], 
                  value_vars=['arc_baseline', 'arc_new-cities', 'arc_expansion', 'arc_unplanned'],
                  var_name='scenario')
highlight_lads = set(df[df['timestep'] == 2050 ].sort_values('value')[-15:]['geo_label'])

In [ ]:
def group_hlads(index):
    if index in highlight_lads:
        return index
    else:
        return 'Other'

In [ ]:
arc_dwellings.head()#.reset_index().set_index(['timestep', 'geo_label']).groupby(by=['geo_label'], observed=True).sum()


# Plot difference spatially

In [ ]:
def plot_difference_spatially(df, arc_lads, remove_2015=True):

    melted_dwellings = reshape(df).drop(columns='geometry')
    
    def log(series):
        return series.apply(lambda x: np.log(x))
    
    melted_dwellings['norm_value'] = log(melted_dwellings['value'])
    
    baseline = melted_dwellings[melted_dwellings['scenario']=='arc_baseline'].drop(columns='scenario')
    non_baseline = melted_dwellings[~(melted_dwellings['scenario']=='arc_baseline')]
    
    diff = non_baseline.set_index(['scenario', 'geo_label', 'timestep']).sub(
                baseline.set_index(['geo_label', 'timestep'])).reset_index()
    diff = highlight(diff)  
    
    diff_spatial = arc_lads.reset_index().set_index('geo_label').join(
                       diff.set_index('geo_label'), rsuffix='bla')
    
    cmap = sbn.diverging_palette(h_neg=220, h_pos=10, s=74, l=50, sep=10, n=9, as_cmap=True)
    
    def plot_diff_map(data, *args, **kwargs):
        cax = plt.gca()
        ax = data.plot(column='norm_value', ax=cax, cmap=cmap, 
                       vmin=-diff_spatial['norm_value'].max(), vmax=diff_spatial['norm_value'].max())
        ax.set_axis_off()
        return ax

    # Remove baseline (no difference across scenarios)
    if remove_2015:
        diff_spatial = diff_spatial[~(diff_spatial['timestep'] == 2015)]

    with sbn.plotting_context('paper'):
        g = sbn.FacetGrid(data=diff_spatial, row='scenario', col='timestep', 
                          height=4, sharex=True, sharey=True)
        g.map_dataframe(plot_diff_map)
        
    return g

In [ ]:
plot_difference_spatially(arc_dwellings, arc_lads).savefig("dwellings_difference.png")

In [ ]:
plot_difference_spatially(arc_population, arc_lads).savefig('population_difference.png')

In [ ]:
plot_difference_spatially(arc_jobs, arc_lads, remove_2015=False).savefig('jobs_difference.png')